## Phase 4: Personalization & Strategy Mapping

###  Objective:
Use churn predictions (`is_at_risk`) and customer segments to craft **targeted re-engagement and loyalty strategies**.

###  Why this matters:
- Not all customers need the same offer
- Personalization improves conversion, retention, and marketing ROI

###  What We'll Do:
1. Reload the saved XGBoost model
2. Generate churn predictions for the full customer base
3. Define strategic actions per customer segment and risk
4. Output a campaign-ready dataset for marketing execution


---

In [96]:
import joblib
import pandas as pd
import sys
import os
import importlib.util

#### 1a. Reload Saved XGBoost Model & Predict Churn

**What this does:**
- Reloads the previously saved model from disk
- Uses the model to predict churn risk (`is_at_risk`) for all customers

**Why this matters:**
- Enables us to personalize actions based on actual risk scores
- Separates model training from prediction, making the pipeline modular

In [97]:
#  Load the final RFM dataset
rfm = pd.read_csv('../data/processed/rfm_table.csv')

#  Reload the saved XGBoost model
xgb_model = joblib.load('../outputs/models/xgb_churn_model.joblib')

#  Predict churn risk
rfm['churn_prediction'] = xgb_model.predict(rfm[['Recency', 'Frequency', 'Monetary']])

#  Preview results
rfm[['CustomerID', 'Segment', 'churn_prediction']].head()

,CustomerID,Segment,churn_prediction
0,0,At-Risk,1
1,1,Recent Low-Spenders,0
2,2,At-Risk,1
3,3,Recent Low-Spenders,0
4,4,Loyal Customers,0


#### Churn Prediction Sample — Strategy Mapping Foundation

| CustomerID | Segment              | Churn Prediction |
|------------|----------------------|------------------|
| 0          | At-Risk              | 1 (Churn Likely) |
| 1          | Recent Low-Spenders  | 0 (Safe)         |
| 2          | At-Risk              | 1 (Churn Likely) |
| 3          | Recent Low-Spenders  | 0 (Safe)         |
| 4          | Loyal Customers      | 0 (Safe)         |

**Key Insights:**

- The `churn_prediction` column correctly aligns with the original segments:
  - **At-Risk customers are flagged as 1** (likely to churn)
  - **Recent Low-Spenders and Loyal Customers are flagged as 0** (not at risk)
- This confirms that your model generalizes well and aligns with your clustering logic.
- You can now personalize marketing strategies based on **both behavioral segment and churn risk**.

---

#### 1b. Map Personalized Marketing Strategies

**What this does:**
- Uses customer `Segment` and `churn_prediction` to define a new column: `marketing_action`
- Personalizes campaigns based on customer behavior and churn risk

**Why this matters:**
- Helps you deliver highly relevant promotions
- Maximizes campaign ROI by targeting the right customers with the right incentives

**Examples of Strategies:**
- **At-Risk + Churned → Win-back campaign**
- **Loyal + Safe → Loyalty reward**
- **Low-Spender + Churned → Discount or limited-time offer**

In [98]:
import os

print("Looking for personalization.py in scripts/:")
print(os.path.exists(os.path.abspath('./scripts/personalization.py')))

Looking for personalization.py in scripts/:
False


In [99]:
personalization_path = os.path.abspath('../scripts/personalization.py')

# Load module dynamically
spec = importlib.util.spec_from_file_location("personalization", personalization_path)
personalization = importlib.util.module_from_spec(spec)
sys.modules["personalization"] = personalization
spec.loader.exec_module(personalization)

#  Assign the function
assign_action = personalization.assign_action
rfm['marketing_action'] = rfm.apply(assign_action, axis=1)
rfm[['CustomerID', 'Segment', 'churn_prediction', 'marketing_action']].head()

,CustomerID,Segment,churn_prediction,marketing_action
0,0,At-Risk,1,Reactivation Email + Big Discount
1,1,Recent Low-Spenders,0,Upsell: Bundle Offer
2,2,At-Risk,1,Reactivation Email + Big Discount
3,3,Recent Low-Spenders,0,Upsell: Bundle Offer
4,4,Loyal Customers,0,Loyalty Bonus + Referral Code


#### Personalized Campaign Strategy Output — Sample

| CustomerID | Segment              | Churn Prediction | Marketing Action                      |
|------------|----------------------|------------------|---------------------------------------|
| 0          | At-Risk              | 1 (Churn Likely) | Reactivation Email + Big Discount     |
| 1          | Recent Low-Spenders  | 0 (Safe)         | Upsell: Bundle Offer                  |
| 2          | At-Risk              | 1 (Churn Likely) | Reactivation Email + Big Discount     |
| 3          | Recent Low-Spenders  | 0 (Safe)         | Upsell: Bundle Offer                  |
| 4          | Loyal Customers      | 0 (Safe)         | Loyalty Bonus + Referral Code         |

**Key Insights:**

- Each customer now has a **targeted marketing action** tailored to their:
  - **Behavioral segment** (RFM-based clustering)
  - **Churn risk prediction** (from XGBoost)
- This enables fully personalized retention, upsell, and loyalty campaigns
- Makes your dataset **actionable for campaign execution or CRM integration**

---

#### 1c. Export Final Campaign Dataset

**What this does:**
- Saves the enriched RFM dataset (with churn predictions and marketing actions) to a CSV file
- Makes the data ready for use in CRM systems, email marketing platforms, or dashboard tools

**File Location:**
- `outputs/reports/final_campaign_actions.csv`

In [100]:
os.makedirs('../outputs/reports', exist_ok=True)

# Export your final RFM + predictions + actions
rfm[['CustomerID', 'Segment', 'churn_prediction', 'marketing_action']].to_csv(
    '../outputs/reports/final_campaign_actions.csv',
    index=False
)

print("File saved to: ../outputs/reports/final_campaign_actions.csv")


File saved to: ../outputs/reports/final_campaign_actions.csv


#### Final Campaign Dataset Exported

The personalized customer marketing plan has been exported successfully:

**Location:** `outputs/reports/final_campaign_actions.csv`

**What's Inside:**
Each row in the CSV contains:

- `CustomerID`: Unique customer ID
- `Segment`: RFM-based behavior cluster
- `churn_prediction`: Model-based churn risk (1 = likely to churn)
- `marketing_action`: Personalized strategy for outreach

**Strategic Value:**
- This file is ready for **marketing teams, CRM integration**, or **automated email pipelines**
- It reflects a complete, data-driven pipeline from raw transactions → segmentation → prediction → personalization